<a href="https://colab.research.google.com/github/maccelarimatt/Secure_Computing_Project/blob/main/SecureProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Secure Computing Project**

## Preliminary Installations

In [ ]:
!pip install -q transformers accelerate bitsandbytes auto-gptq peft
!pip install -U bitsandbytes
!pip install --no-cache-dir bitsandbytes
!pip install --no-cache-dir bitsandbytes
!nvcc --version


nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


## Setting Up Prompts

In [ ]:
# Define example prompts for this model
code_vuln_snippet = """\
String productId = request.getParameter("id");
String query = "SELECT * FROM products WHERE id = " + productId;
Statement st = connection.createStatement();
ResultSet rs = st.executeQuery(query);
while (rs.next()) {
    String name = rs.getString("name");
    String description = rs.getString("description");
    // display product details
}"""
prompt_vuln_zeroshot = f"Analyze the following Java code for any security vulnerabilities:\n```java\n{code_vuln_snippet}\n```\nWhat vulnerabilities, if any, are present?"
prompt_vuln_cot = f"Analyze the following Java code for security issues. Explain your reasoning step by step, then give a conclusion.\n```java\n{code_vuln_snippet}\n```\nLet's think this through step-by-step."
code_func_snippet = """\
public static void storePasswordInsecurely(String username, String password) {
    // This method is intended to securely store a password, but is it actually secure?
    try {
        Connection conn = DriverManager.getConnection("jdbc:mysql://localhost:3306/mydb", "username", "password");
        String query = "INSERT INTO users (username, password) VALUES (?, ?)";
        PreparedStatement pstmt = conn.prepareStatement(query);
        pstmt.setString(1, username);
        pstmt.setString(2, password); // storing plain password
        pstmt.executeUpdate();
        pstmt.close();
        conn.close();
    } catch (SQLException e) {
        e.printStackTrace();
    }
}"""
prompt_func_zeroshot = f"The following Java method is intended to securely store a user's password in a database. Does it accomplish this goal? Answer briefly.\n```java\n{code_func_snippet}\n```"
prompt_func_cot = f"Does the following Java method correctly implement secure password storage as intended? Please reason through the code step by step and then conclude.\n```java\n{code_func_snippet}\n```\nLet's analyze it step-by-step."


## Model 1 - Code Llama 7B Instruct

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
import torch
import gc
import os
import shutil
model_name = "codellama/CodeLlama-7b-Instruct-hf"
quant_config = BitsAndBytesConfig(load_in_4bit=True)

# Use 4-bit quantization and float16 for speed and reduced memory footprint.
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained( # ISSUES WITH IT BEING A GATED REPO
    model_name,
    device_map="auto",
    quantization_config=quant_config,
    trust_remote_code=True,
    torch_dtype=torch.float16,
)

# Create a pipeline for text generation
text_gen_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

# -----------------------------
# 3) Helper function to generate text from a prompt
# -----------------------------
def generate_response(prompt, pipe, max_length=512):
    sequences = pipe(
        prompt,
        do_sample=True,
        top_k=10,
        temperature=0.1,
        top_p=0.95,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        max_length=max_length,
    )
    return sequences[0]["generated_text"]

# -----------------------------
# 4) Run each prompt and print responses
# -----------------------------
print("**Vulnerability Detection (Zero-shot)**")
print(generate_response(prompt_vuln_zeroshot, text_gen_pipeline))

print("\n**Vulnerability Detection (Chain-of-Thought)**")
print(generate_response(prompt_vuln_cot, text_gen_pipeline))

print("\n**Functional Validation (Zero-shot)**")
print(generate_response(prompt_func_zeroshot, text_gen_pipeline))

print("\n**Functional Validation (Chain-of-Thought)**")
print(generate_response(prompt_func_cot, text_gen_pipeline))

# -----------------------------
# 5) Cleanup: Remove downloaded caches and free GPU memory
# -----------------------------
def cleanup_all():
    """
    Deletes cache directories used by transformers and torch,
    runs garbage collection, and empties the CUDA cache.
    WARNING: This will remove downloaded model and tokenizer caches.
    """
    gc.collect()
    torch.cuda.empty_cache()
    cache_dirs = [
        os.path.expanduser("~/.cache/huggingface/transformers"),
        os.path.expanduser("~/.cache/huggingface/hub"),
        os.path.expanduser("~/.cache/torch")
    ]
    for cache_dir in cache_dirs:
        if os.path.exists(cache_dir):
            try:
                shutil.rmtree(cache_dir)
                print(f"Deleted cache directory: {cache_dir}")
            except Exception as e:
                print(f"Could not delete {cache_dir}: {e}")
        else:
            print(f"Cache directory does not exist: {cache_dir}")

# Uncomment the line below to clean up caches after execution
cleanup_all()

ImportError: Using `bitsandbytes` 4-bit quantization requires the latest version of bitsandbytes: `pip install -U bitsandbytes`

## Model 2 - Llama 2 7B Chat

In [ ]:
model_name = "meta-llama/Llama-2-7b-chat-hf"  # Hugging Face format of Llama2 7B Chat
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    load_in_4bit=True
)

def generate_response_llama(prompt, max_new_tokens=256):
    # Llama-2-Chat expects a conversation format. We can simulate a single-turn.
    # Format: <s><</SYS>> system_message <</SYS>> user_message <s>
    system_msg = "You are a helpful security code analysis assistant."
    user_msg = prompt
    formatted_input = f"<s>[INST] <<SYS>>\n{system_msg}\n<</SYS>>\n\n{user_msg} [/INST]"
    inputs = tokenizer(formatted_input, return_tensors='pt').to(model.device)
    outputs = model.generate(**inputs, max_new_tokens=max_new_tokens,
                              do_sample=False, temperature=0.2)
    result = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return result

# Reuse the same prompts defined earlier for vulnerability and function tasks:
print("**Vulnerability Detection (Zero-shot)**")
print(generate_response_llama(prompt_vuln_zeroshot))
print("\n**Vulnerability Detection (Chain-of-Thought)**")
print(generate_response_llama(prompt_vuln_cot))
print("\n**Functional Validation (Zero-shot)**")
print(generate_response_llama(prompt_func_zeroshot))
print("\n**Functional Validation (Chain-of-Thought)**")
print(generate_response_llama(prompt_func_cot))

# Cleanup
del model, tokenizer
gc.collect()
torch.cuda.empty_cache()
print("\n[Model memory cleared]")


OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/meta-llama/Llama-2-7b-chat-hf.
401 Client Error. (Request ID: Root=1-67e1a30c-7143682913fa28191dde1237;40cae2e5-e905-46df-abed-fe4138623880)

Cannot access gated repo for url https://huggingface.co/meta-llama/Llama-2-7b-chat-hf/resolve/main/config.json.
Access to model meta-llama/Llama-2-7b-chat-hf is restricted. You must have access to it and be authenticated to access it. Please log in.

## Model 3 - StarCoder2 15B

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_name = "bigcode/starcoder2-7b"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    load_in_4bit=True
)

def generate_response_starcoder(prompt, max_new_tokens=256):
    # StarCoder base is not instruction-tuned, so include a brief system hint:
    tech_assistant_prefix = "'''[System: You are an AI assistant skilled in code analysis.]'''\n"
    # The triple quotes and System label above are to nudge the model into explanatory mode.
    inputs = tokenizer(tech_assistant_prefix + prompt, return_tensors='pt').to(model.device)
    outputs = model.generate(**inputs, max_new_tokens=max_new_tokens,
                              do_sample=False, temperature=0.2)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

print("**Vulnerability Detection (Zero-shot)**")
print(generate_response_starcoder(prompt_vuln_zeroshot))
print("\n**Vulnerability Detection (Chain-of-Thought)**")
print(generate_response_starcoder(prompt_vuln_cot))
print("\n**Functional Validation (Zero-shot)**")
print(generate_response_starcoder(prompt_func_zeroshot))
print("\n**Functional Validation (Chain-of-Thought)**")
print(generate_response_starcoder(prompt_func_cot))

# Cleanup
del model, tokenizer
gc.collect()
torch.cuda.empty_cache()
print("\n[Model memory cleared]")


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


**Vulnerability Detection (Zero-shot)**


/usr/local/lib/python3.11/dist-packages/bitsandbytes/nn/modules.py:451: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:629: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


'''[System: You are an AI assistant skilled in code analysis.]'''
Analyze the following Java code for any security vulnerabilities:
```java
String productId = request.getParameter("id");
String query = "SELECT * FROM products WHERE id = " + productId;
Statement st = connection.createStatement();
ResultSet rs = st.executeQuery(query);
while (rs.next()) {
    String name = rs.getString("name");
    String description = rs.getString("description");
    // display product details
}
```
What vulnerabilities, if any, are present?

**Answer:**

The code is vulnerable to SQL injection.

**Explanation:**

The `productId` variable is not properly sanitized.

**Solution:**

Sanitize the `productId` variable before using it in the query.

**Example:**

```java
String productId = request.getParameter("id");
productId = productId.replaceAll("[^0-9]", "");
String query = "SELECT * FROM products WHERE id = " + productId;
Statement st = connection.createStatement();
ResultSet rs = st.executeQuery(query

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


'''[System: You are an AI assistant skilled in code analysis.]'''
Analyze the following Java code for security issues. Explain your reasoning step by step, then give a conclusion.
```java
String productId = request.getParameter("id");
String query = "SELECT * FROM products WHERE id = " + productId;
Statement st = connection.createStatement();
ResultSet rs = st.executeQuery(query);
while (rs.next()) {
    String name = rs.getString("name");
    String description = rs.getString("description");
    // display product details
}
```
Let's think this through step-by-step.

First, we get the product ID from the request. This is a string, so we need to make sure it's a valid ID.

Second, we build the query string. We know that the product ID is a string, so we can concatenate it directly.

Third, we create a statement and execute the query.

Fourth, we iterate through the result set. We know that the product ID is a string, so we can concatenate it directly.

Fifth, we display the product det

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


'''[System: You are an AI assistant skilled in code analysis.]'''
The following Java method is intended to securely store a user's password in a database. Does it accomplish this goal? Answer briefly.
```java
public static void storePasswordInsecurely(String username, String password) {
    // This method is intended to securely store a password, but is it actually secure?
    try {
        Connection conn = DriverManager.getConnection("jdbc:mysql://localhost:3306/mydb", "username", "password");
        String query = "INSERT INTO users (username, password) VALUES (?,?)";
        PreparedStatement pstmt = conn.prepareStatement(query);
        pstmt.setString(1, username);
        pstmt.setString(2, password); // storing plain password
        pstmt.executeUpdate();
        pstmt.close();
        conn.close();
    } catch (SQLException e) {
        e.printStackTrace();
    }
}
```

'''[System: You are an AI assistant skilled in code analysis.]'''
The following Java method is intended to

## Model 4 - WizardCoder 7B

In [ ]:
model_name = "WizardLMTeam/WizardCoder-Python-13B-V1.0"  # (hypothetical name for a 7B general WizardCoder)
# If the above is not available, one could use WizardCoder-15B-v1.0 with 4-bit as an alternative.
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    load_in_4bit=True,
    trust_remote_code=True
)

def generate_response_wizard(prompt, max_new_tokens=256):
    inputs = tokenizer(prompt, return_tensors='pt').to(model.device)
    outputs = model.generate(**inputs, max_new_tokens=max_new_tokens,
                              do_sample=False, temperature=0.2)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

print("**Vulnerability Detection (Zero-shot)**")
print(generate_response_wizard(prompt_vuln_zeroshot))
print("\n**Vulnerability Detection (Chain-of-Thought)**")
print(generate_response_wizard(prompt_vuln_cot))
print("\n**Functional Validation (Zero-shot)**")
print(generate_response_wizard(prompt_func_zeroshot))
print("\n**Functional Validation (Chain-of-Thought)**")
print(generate_response_wizard(prompt_func_cot))

# Cleanup
del model, tokenizer
gc.collect()
torch.cuda.empty_cache()
print("\n[Model memory cleared]")


## Model 5 - Mistral 7B Instruct

In [ ]:
model_name = "mistralai/Mistral-7B-Instruct-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    load_in_4bit=True,
    trust_remote_code=True
)

def format_mistral_prompt(user_prompt):
    # Mistral instruct requires wrapping the user prompt in [INST] tags, and starting with <s>
    return f"<s>[INST] {user_prompt} [/INST]"

def generate_response_mistral(user_prompt, max_new_tokens=256):
    prompt = format_mistral_prompt(user_prompt)
    inputs = tokenizer(prompt, return_tensors='pt').to(model.device)
    outputs = model.generate(**inputs, max_new_tokens=max_new_tokens,
                              do_sample=False, temperature=0.2)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

print("**Vulnerability Detection (Zero-shot)**")
print(generate_response_mistral("Analyze the following Java code for security vulnerabilities:\n```java\n" + code_vuln_snippet + "\n```"))
print("\n**Vulnerability Detection (Chain-of-Thought)**")
print(generate_response_mistral("Analyze the Java code below for vulnerabilities. Think step by step and explain your reasoning.\n```java\n" + code_vuln_snippet + "\n```"))
print("\n**Functional Validation (Zero-shot)**")
print(generate_response_mistral("The following code is meant to securely store a password. Does it do so successfully?\n```java\n" + code_func_snippet + "\n```"))
print("\n**Functional Validation (Chain-of-Thought)**")
print(generate_response_mistral("Is the following Java function properly implementing secure password storage as intended? Provide a step-by-step analysis and conclusion.\n```java\n" + code_func_snippet + "\n```"))

# Cleanup
del model, tokenizer
gc.collect()
torch.cuda.empty_cache()
print("\n[Model memory cleared]")


tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

[INST] Analyze the following Java code for security vulnerabilities:
```java
String productId = request.getParameter("id");
String query = "SELECT * FROM products WHERE id = " + productId;
Statement st = connection.createStatement();
ResultSet rs = st.executeQuery(query);
while (rs.next()) {
    String name = rs.getString("name");
    String description = rs.getString("description");
    // display product details
}
``` [/INST] The given Java code is vulnerable to SQL injection attacks. The `productId` variable is directly concatenated into the SQL query without any sanitization or validation. This means that an attacker can inject malicious SQL code into the `productId` parameter and potentially gain access to sensitive data or perform unauthorized actions on the database.

To mitigate this vulnerability, the `productId` variable should be sanitized and validated before being used in the SQL query. One way to do this is to use parameterized queries, which allow you to specify placehol

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[INST] Analyze the Java code below for vulnerabilities. Think step by step and explain your reasoning.
```java
String productId = request.getParameter("id");
String query = "SELECT * FROM products WHERE id = " + productId;
Statement st = connection.createStatement();
ResultSet rs = st.executeQuery(query);
while (rs.next()) {
    String name = rs.getString("name");
    String description = rs.getString("description");
    // display product details
}
``` [/INST] This Java code appears to be vulnerable to SQL injection attacks. Here's a step-by-step analysis of the code:

1. The `productId` variable is set to the value of the `id` parameter from the HTTP request. This is a good practice, as it ensures that the product ID is properly sanitized and validated before being used in the SQL query.
2. The SQL query is constructed by concatenating the `productId` variable with a string literal. This is where the vulnerability lies. If the `productId` variable contains malicious SQL code, it can 

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[INST] The following code is meant to securely store a password. Does it do so successfully?
```java
public static void storePasswordInsecurely(String username, String password) {
    // This method is intended to securely store a password, but is it actually secure?
    try {
        Connection conn = DriverManager.getConnection("jdbc:mysql://localhost:3306/mydb", "username", "password");
        String query = "INSERT INTO users (username, password) VALUES (?, ?)";
        PreparedStatement pstmt = conn.prepareStatement(query);
        pstmt.setString(1, username);
        pstmt.setString(2, password); // storing plain password
        pstmt.executeUpdate();
        pstmt.close();
        conn.close();
    } catch (SQLException e) {
        e.printStackTrace();
    }
}
``` [/INST] No, this code does not securely store a password. The password is stored in plain text in the database, which is not secure. A secure way to store a password would be to hash it using a strong hashing algor

## Garbage Deletion Function

In [2]:
import gc
import torch
import os
import shutil

def cleanup_all():
    """
    Frees up GPU memory and deletes local cache directories used by transformers and torch.
    WARNING: This will remove local cache directories (e.g. models and tokenizers)
             so they will need to be re-downloaded if needed later.
    """
    gc.collect()
    torch.cuda.empty_cache()

    cache_dirs = [
        os.path.expanduser("~/.cache/huggingface/transformers"),
        os.path.expanduser("~/.cache/huggingface/hub"),
        os.path.expanduser("~/.cache/torch")
    ]

    for cache_dir in cache_dirs:
        if os.path.exists(cache_dir):
            try:
                shutil.rmtree(cache_dir)
                print(f"Deleted cache directory: {cache_dir}")
            except Exception as e:
                print(f"Could not delete {cache_dir}: {e}")
        else:
            print(f"Cache directory does not exist: {cache_dir}")

    print("All caches and memory cleared.")

cleanup_all()


Cache directory does not exist: /root/.cache/huggingface/transformers
Cache directory does not exist: /root/.cache/huggingface/hub
Cache directory does not exist: /root/.cache/torch
All caches and memory cleared.
